# Setup

## Imports & `.env` Setup

In [1]:
import pandas as pd, numpy as np, tweepy as tw, streamlit as st
import requests, os, dotenv, json
dotenv.load_dotenv();

## Load Secret Variables

### Twitter

In [2]:
def do_tweepy_auth():
    key, secret_key = (os.getenv(i) for i in ["TWEET_KEY", "TWEET_SECRET_KEY"])
    auth = tw.OAuthHandler(key, secret_key)
    api = tw.API(auth, wait_on_rate_limit=True)

### Airtable

In [3]:
token, base, url = (os.getenv(i) for i in ["AIRTABLE_TOKEN",
                                           "AIRTABLE_BASE_ID",
                                           "AIRTABLE_URL"])

## Functions

### Twitter

In [5]:
def getTweets(search, method = "search_tweets", n = 50):
    tweets = tw.Cursor(eval(f"api.{method}"),
              q=search,
              lang="en").items(n)
    tweets = list(tweets)
    print("Total Tweets fetched:", len(tweets))
    return tweets

## Airtable

In [6]:
def post(data, table = "ratings"):
    """Add data to the Airtable."""
    sheet = f"{url}/{table}"
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    response = requests.request("POST", sheet, headers=headers, data=json.dumps(data))
    return response

In [7]:
def get(query = None, table = "tweets"):
    """Retrieve Tweet Data"""
    sheet = f"{url}/{table}"
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    response = requests.request("GET", sheet, headers=headers)
    data = [{"id" : record["id"]} | record["fields"] for record in eval(response.text)["records"]]
    frame = pd.DataFrame(data)
    return frame

In [8]:
def update(username, table = "tweets", step = 1):
    """Update specific field values for a record."""
    sheet = f"{url}/{table}"
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    record = dict(get().query(f"user == '{username}'").iloc[0, :])
    new_record = {"records": [ {"id": record["id"], "fields": {"Rated": int(record["Rated"]) + step}}]}
    response = requests.request("PATCH", sheet, headers=headers, data=json.dumps(new_record))
    return response

# Work

In [9]:
def getUser(username):
    return api.search_users(username, count = 1)
def getTimeline(username):
    tweets = api.search_tweets(username, count = 10000)
    cleanTweets = [tweet for tweet in tweets if tweet.user._json["screen_name"] == username]
    return cleanTweets